In [3]:
import pandas as pd
import numpy as np
import math

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score, RocCurveDisplay, roc_curve, auc

# Chicago Car Crash Analysis

## Overview 
add overview 

## Business Problem

add buisness problem

## Data Source 

Car crash and vehicle accident data was sourced from the [Chicago Data Portal](https://data.cityofchicago.org/Transportation/Traffic-Crashes-People/u6pd-qa9d/about_data). It originates from the [City of Chicago](http://www.chicago.gov/)

Data collection began in 2015, but was only adopted by all police departments in 2017. The most recent data entry was from September 26, 2024. 

## Results

add results

# Data Import and Cleaning 

In [24]:
df_vehicles = pd.read_csv("data/Traffic_Crashes_Vehicles.csv")
df_people = pd.read_csv("data/Traffic_Crashes_People.csv")
df_crashes = pd.read_csv("data/Traffic_Crashes_Crashes.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'data/Traffic_Crashes_Vehicles.csv'

## Merge Dataframes

First we sample 300,000 crash reports, then we merge the vehicle and people involved. 

In [7]:
#Random sample of vehicles 
df_accidents = df_vehicles.sample(n=300000, random_state=42)

NameError: name 'df_vehicles' is not defined

In [8]:
df_accidents.info()

NameError: name 'df_accidents' is not defined

In [9]:
#Then we can add the crash report data 
df_accidents = df_accidents.merge(df_crashes, how="inner", on=["CRASH_RECORD_ID", "CRASH_DATE"])

NameError: name 'df_accidents' is not defined

In [10]:
df_accidents.info()

NameError: name 'df_accidents' is not defined

In [11]:
#every person and vehicle should have a match for the record ID, vehicle ID, and crash date
#This makes the dataframe larger as multiple people are involved in each accident 
df_accidents = df_accidents.merge(df_people, how="inner", on=["CRASH_RECORD_ID", "VEHICLE_ID", "CRASH_DATE"])

NameError: name 'df_accidents' is not defined

In [12]:
df_accidents.info()

NameError: name 'df_accidents' is not defined

The dataframe gets larger when merging in the people, as there are sometimes more than one person involved in an accident.  

# THIS IS FOR WORKING PURPOSES. 

# USE THIS TO IMPORT DF WITHOUT RUNNING THE CELLS ABOVE.

## Export Final

In [13]:
df_accidents.to_csv('data/Final_Data.csv')

## Read Final

In [14]:
df_accidents = pd.read_csv("data/Final_Data.csv")

# ^^^^^ DELETE BEFORE SUBMISSION

In [15]:
#have the crash date as a datetime object. 
df_accidents['CRASH_DATE'] = pd.to_datetime(df_accidents['CRASH_DATE'], format="%d%m%Y %I:%M:%S %p")

df_accidents.info()

ValueError: time data '05/24/2022 08:23:00 PM' does not match format '%d%m%Y %I:%M:%S %p' (match)

## Drop Unnessisary Columns

In [16]:
#remove columns with less than 50% of data present 
df_accidents.dropna(axis=1, thresh = int(0.5*df_accidents.shape[0]), inplace=True)

In [18]:
cols_to_drop = [
#remove unessisary id columns now the data is joined 
'VEHICLE_ID', 'CRASH_RECORD_ID', 'CRASH_UNIT_ID',
#drop LAT and LONG (we have location)
'LATITUDE', 'LONGITUDE',
#...also drop streets and directions. 
'STREET_NO', 'STREET_DIRECTION', 'STREET_NAME', 'TRAVEL_DIRECTION', 
#police specific 
'BEAT_OF_OCCURRENCE', 
#everything is chicago 
'CITY', 'ZIPCODE', 
#covered by MOST_SEVERE_INJURY
'INJURY_CLASSIFICATION',
#many NAN
'INJURIES_UNKNOWN'
]

df_accidents.drop(columns=cols_to_drop, inplace=True)


KeyError: "['VEHICLE_ID' 'CRASH_RECORD_ID' 'CRASH_UNIT_ID' 'LATITUDE' 'LONGITUDE'\n 'STREET_NO' 'STREET_DIRECTION' 'STREET_NAME' 'TRAVEL_DIRECTION'\n 'BEAT_OF_OCCURRENCE' 'CITY' 'ZIPCODE' 'INJURY_CLASSIFICATION'\n 'INJURIES_UNKNOWN'] not found in axis"

In [19]:
#map the injuries to a number 

def weather_code(weather):
    """
    Codes are based on https://openweathermap.org/weather-conditions
    0 - Unknown 
    500 - Rain
    600 - Snow
    700 - 'Atmosphere' (Fog, Dust, Haze) 
    800 - Clear
    801 - Clouds 

    """
    if weather == 'CLEAR':
        return 800
    elif weather == 'CLOUDY/OVERCAST':
        return 801
    elif weather == 'RAIN':
        return 500
    elif weather == 'SNOW' or weather == 'FREEZING RAIN/DRIZZLE' or weather== 'SLEET/HAIL' or weather == 'BLOWING SNOW':
        return 600
    elif weather == 'FOG/SMOKE/HAZE' or weather == 'BLOWING SAND/SOIL/DIRT':
        return 700
    else:
        return 0
    
df_accidents['WEATHER_CONDITION'] = df_accidents['WEATHER_CONDITION'].map(weather_code)

In [20]:
#map the injuries to a number 

def injury_severity(severity):
    """
    Injuties are categorized on a scale of 0 to X
    0 - No Indication of Injury 
    1 - REPORTED, NOT EVIDENT 
    2 - NONincapacitating Injury 
    3 - incapacitating Injury 
    4 - Fatal Injury 
    """
    if severity == 'NO INDICATION OF INJURY':
        return 0
    elif severity == 'REPORTED, NOT EVIDENT':
        return 1
    elif severity == 'NONINCAPACITATING INJURY':
        return 2
    elif severity == 'INCAPACITATING INJURY':
        return 3
    elif severity == 'FATAL':
        return 4
    else:
        return 0
    

df_accidents['INJURY_CLASS'] = df_accidents['MOST_SEVERE_INJURY'].map(injury_severity)

In [21]:
#map the injuries to a number 

def crash_severity(severity):
    """
    Injuties are categorized on a scale of 0 to X
    0 - No Injury / Drive Away 
    1 - Injury and / or tow  
    """
    if severity == 'NO INJURY / DRIVE AWAY':
        return 0
    else:
        return 1
    

df_accidents['CRASH_TYPE'] = df_accidents['CRASH_TYPE'].map(crash_severity)

In [22]:
#drop the columns 
in_cols_to_drop = [
#remove counts of each injury 
'INJURIES_FATAL', 'INJURIES_INCAPACITATING', 'INJURIES_NON_INCAPACITATING',
'INJURIES_REPORTED_NOT_EVIDENT', 'INJURIES_NO_INDICATION'
]

df_accidents.drop(columns=in_cols_to_drop, inplace=True)

KeyError: "['INJURIES_FATAL' 'INJURIES_INCAPACITATING' 'INJURIES_NON_INCAPACITATING'\n 'INJURIES_REPORTED_NOT_EVIDENT' 'INJURIES_NO_INDICATION'] not found in axis"

In [23]:
count = 0
for column in df_accidents.iloc[0]:
    print(df_accidents.columns[count], ":", column, ",", df_accidents.iloc[1].iloc[count])
    count += 1

Unnamed: 0 : 0 , 1
Unnamed: 0.1 : 0 , 1
Unnamed: 0.1.1 : 0 , 1
Unnamed: 0.1.1.1 : 0 , 1
CRASH_DATE : 05/24/2022 08:23:00 PM , 09/28/2019 01:45:00 PM
UNIT_NO : 1 , 2
UNIT_TYPE : DRIVER , DRIVER
MAKE : TOYOTA , VOLKSWAGEN
MODEL : CAMRY , JETTA
LIC_PLATE_STATE : IL , IL
VEHICLE_YEAR : 2018.0 , 2012.0
VEHICLE_DEFECT : UNKNOWN , UNKNOWN
VEHICLE_TYPE : PASSENGER , PASSENGER
VEHICLE_USE : PERSONAL , PERSONAL
MANEUVER : TURNING LEFT , STRAIGHT AHEAD
OCCUPANT_CNT : 1.0 , 1.0
FIRST_CONTACT_POINT : FRONT , ROOF
POSTED_SPEED_LIMIT_x : 30 , 20
TRAFFIC_CONTROL_DEVICE_x : STOP SIGN/FLASHER , NO CONTROLS
DEVICE_CONDITION_x : FUNCTIONING PROPERLY , NO CONTROLS
WEATHER_CONDITION_x : CLEAR , CLEAR
LIGHTING_CONDITION_x : DUSK , DAYLIGHT
FIRST_CRASH_TYPE_x : PEDALCYCLIST , SIDESWIPE OPPOSITE DIRECTION
TRAFFICWAY_TYPE_x : FOUR WAY , OTHER
ALIGNMENT_x : STRAIGHT AND LEVEL , STRAIGHT AND LEVEL
ROADWAY_SURFACE_COND_x : DRY , DRY
ROAD_DEFECT_x : NO DEFECTS , UNKNOWN
REPORT_TYPE_x : ON SCENE , NOT ON SCENE (DESK

In [24]:
matrix = df_accidents.corr(numeric_only=True)
print(matrix)

TypeError: corr() got an unexpected keyword argument 'numeric_only'

In [25]:
plt.figure(figsize=(30, 30))

#create a mask to remove the duplicate upper half
mask = np.triu(np.ones_like(matrix, dtype=np.bool))

heatmap = sns.heatmap(matrix, vmin=-1, mask=mask, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12);

NameError: name 'matrix' is not defined

<Figure size 2160x2160 with 0 Axes>

Use a Decsicion Tree model to determine importaint categories. 

### DT

In [26]:
df_accidents.PRIM_CONTRIBUTORY_CAUSE.unique()

array(['UNABLE TO DETERMINE', 'DISREGARDING TRAFFIC SIGNALS',
       'IMPROPER LANE USAGE', 'FOLLOWING TOO CLOSELY',
       'IMPROPER TURNING/NO SIGNAL', 'IMPROPER OVERTAKING/PASSING',
       'FAILING TO YIELD RIGHT-OF-WAY',
       'DISREGARDING OTHER TRAFFIC SIGNS', 'DISREGARDING STOP SIGN',
       'FAILING TO REDUCE SPEED TO AVOID CRASH', 'NOT APPLICABLE',
       'OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MANNER',
       'IMPROPER BACKING', 'DISTRACTION - FROM INSIDE VEHICLE',
       'DRIVING SKILLS/KNOWLEDGE/EXPERIENCE',
       'ROAD CONSTRUCTION/MAINTENANCE',
       'EXCEEDING AUTHORIZED SPEED LIMIT',
       'DISTRACTION - FROM OUTSIDE VEHICLE', 'WEATHER',
       'DRIVING ON WRONG SIDE/WRONG WAY',
       'UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE WHEN ARREST IS EFFECTED)',
       'PHYSICAL CONDITION OF DRIVER', 'EQUIPMENT - VEHICLE CONDITION',
       'CELL PHONE USE OTHER THAN TEXTING', 'DISREGARDING ROAD MARKINGS',
       'VISION OBSCURED (SIGNS, TR

In [29]:
#map the primary contributory causes to a number 

def primary_cause(cause):
    
    
    if cause == 'UNABLE TO DETERMINE':
        return 0
    elif cause == 'DISREGARDING TRAFFIC SIGNALS':
        return 1
    elif cause == 'IMPROPER LANE USAGE':
        return 2
    elif cause == 'FOLLOWING TOO CLOSELY':
        return 3
    elif cause == 'IMPROPER TURNING/NO SIGNAL':
        return 4
    elif cause == 'IMPROPER OVERTAKING/PASSING':
        return 5
    elif cause == 'FAILING TO YIELD RIGHT-OF-WAY':
        return 6
    elif cause == 'DISREGARDING OTHER TRAFFIC SIGNS':
        return 7
    elif cause == 'DISREGARDING STOP SIGN':
        return 8
    elif cause == 'FAILING TO REDUCE SPEED TO AVOID CRASH':
        return 9
    elif cause == 'NOT APPLICABLE':
        return 10
    elif cause == 'OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MANNER':
        return 11
    elif cause == 'IMPROPER BACKING':
        return 12
    elif cause == 'DISTRACTION - FROM INSIDE VEHICLE':
        return 13
    elif cause == 'DRIVING SKILLS/KNOWLEDGE/EXPERIENCE':
        return 14
    elif cause == 'ROAD CONSTRUCTION/MAINTENANCE':
        return 15
    elif cause == 'EXCEEDING AUTHORIZED SPEED LIMIT':
        return 16
    elif cause == 'DISTRACTION - FROM OUTSIDE VEHICLE':
        return 17
    elif cause == 'WEATHER':
        return 18
    elif cause == 'DRIVING ON WRONG SIDE/WRONG WAY':
        return 19
    elif cause == 'UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE WHEN ARREST IS EFFECTED)':
        return 20
    elif cause == 'PHYSICAL CONDITION OF DRIVER':
        return 21
    elif cause == 'EQUIPMENT - VEHICLE CONDITION':
        return 22
    elif cause == 'CELL PHONE USE OTHER THAN TEXTING':
        return 23
    elif cause == 'DISREGARDING ROAD MARKINGS':
        return 24
    elif cause == 'VISION OBSCURED (SIGNS, TREE LIMBS, BUILDINGS, ETC.)':
        return 25
    elif cause == 'ROAD ENGINEERING/SURFACE/MARKING DEFECTS':
        return 26
    elif cause == 'TEXTING':
        return 27
    elif cause == 'EVASIVE ACTION DUE TO ANIMAL, OBJECT, NONMOTORIST':
        return 28
    elif cause == 'EXCEEDING SAFE SPEED FOR CONDITIONS':
        return 29
    elif cause == 'DISTRACTION - OTHER ELECTRONIC DEVICE (NAVIGATION DEVICE, DVD PLAYER, ETC.)':
        return 30
    elif cause == 'HAD BEEN DRINKING (USE WHEN ARREST IS NOT MADE)':
        return 31
    elif cause == 'DISREGARDING YIELD SIGN':
        return 32
    elif cause == 'TURNING RIGHT ON RED':
        return 33
    elif cause == 'ANIMAL':
        return 34
    elif cause == 'OBSTRUCTED CROSSWALKS':
        return 35
    elif cause == 'BICYCLE ADVANCING LEGALLY ON RED LIGHT':
        return 36
    elif cause == 'PASSING STOPPED SCHOOL BUS':
        return 37
    elif cause == 'RELATED TO BUS STOP':
        return 38
    elif cause == 'MOTORCYCLE ADVANCING LEGALLY ON RED LIGHT':
        return 39
    else:
        return 0
    

df_accidents['PRIM_CONTRIBUTORY_CAUSE'] = df_accidents['PRIM_CONTRIBUTORY_CAUSE'].map(primary_cause)

# EDA

figures?

## Combine Columns into Categories 

In [17]:
#py

## Train Test Split

# Model 1

Justification of Model 1

Conclusions from model 1.

# Model 2

Justification of Model 2

Conclusions from model 1.

# Conclusions 

# Next Steps